In [9]:
from __future__ import division
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import average_precision_score
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


#Initialize stop words for Bahasa Indonesia
id_stop_words_file = pd.read_csv('id-stopwords.txt', header=None, names=['stopwords'])
id_stop_words_list = []
for i in range(len(id_stop_words_file)):
    id_stop_words_list.append(id_stop_words_file.values[i][0])

#Initialize Dataset
dataset = pd.read_csv('rangkuman-saja.csv', encoding='latin', header=0, sep=',')
input = dataset['judul_proposal'].str.lower().str.replace('[^a-zA-Z0-9 ]', '')
target = dataset['deskripsi'].str.lower()

#See dataset distribution
print(dataset.deskripsi.value_counts())


#Initialize Stemmer, Stemmernya cukup lama jadi comment aja biar cepet
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()
# for i in range(len(input)):
#    print(i)
#    input[i] = stemmer.stem(input[i])

#Initialize TFIDF Vectorizer
tvect = TfidfVectorizer(min_df=1,stop_words=id_stop_words_list)

#Split Test dan Data Train
x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=4)



tvect1 = TfidfVectorizer(min_df=1,stop_words=id_stop_words_list)
#tvect1 = CountVectorizer()
x_traincv=tvect1.fit_transform(x_train)
x_testcv=tvect1.transform(x_test)
x_traincv=tvect1.fit_transform(x_train)
x_testcv=tvect1.transform(x_test)

#Oversampling data use SMOTE
sm = SMOTE(random_state=4, ratio=1)
# sm = ADASYN()
x_train_res, y_train_res = sm.fit_sample(x_traincv, y_train)

# model = RandomForestClassifier(n_estimators=50) #around 54%
model = LinearSVC() #around 67%
# model = LinearSVC(loss='hinge', multi_class='ovr') # 51%

# y_train = y_train_res.astype('str')
# model.fit(x_traincv,y_train)

y_train_res = y_train_res.astype('str')
model.fit(x_train_res,y_train_res)

print("Cross Validation")
predictions=model.predict(x_testcv)
count = 0
for i in range (len(predictions)):
  if predictions[i]==y_test.values[i]:
     count = count + 1
print(count / len(y_test))

#Evaluation Metrics
print(accuracy_score(y_test, predictions))
print(precision_score(y_test, predictions, average=None))
print(recall_score(y_test, predictions, average=None))  
print(confusion_matrix(y_test, predictions))
print(f1_score(y_test, predictions, average=None))
print(f1_score(y_test, predictions, average='micro'))

print("Same Data")
predictions=model.predict(x_traincv)
count = 0
for i in range (len(predictions)):
  # print(predictions[i])
  # print(y_train.values[i])
  if predictions[i]==y_train.values[i]:
     count = count + 1
print(count / len(y_train))




#SAVE MODEL
# save the model to disk
filename = 'KlasifikasiPakar.sav'
pickle.dump(model, open(filename, 'wb'))

intelligent system               81
software engineering             73
sistem komputer                  70
information management           67
enterprise computing             52
social and professional issue    38
hardware                         27
graphics and visualization       25
algoritma dan pemrograman        22
dasar matematika                  8
Name: deskripsi, dtype: int64


/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


Cross Validation
0.6559139784946236
0.6559139784946236
[0.375      0.         0.57142857 1.         0.83333333 0.57894737
 0.625      0.73333333 1.         0.6       ]
[1.         0.         0.72727273 0.33333333 0.83333333 0.6875
 0.83333333 0.91666667 0.63636364 0.27272727]
[[ 3  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  1  0  0  0]
 [ 0  0  8  0  0  2  1  0  0  0]
 [ 1  0  1  3  1  2  0  1  0  0]
 [ 1  0  0  0  5  0  0  0  0  0]
 [ 0  0  2  0  0 11  2  0  0  1]
 [ 0  0  0  0  0  2 10  0  0  0]
 [ 0  0  1  0  0  0  0 11  0  0]
 [ 0  0  0  0  0  0  0  3  7  1]
 [ 2  0  2  0  0  2  2  0  0  3]]
[0.54545455 0.         0.64       0.5        0.83333333 0.62857143
 0.71428571 0.81481481 0.77777778 0.375     ]
0.6559139784946236
Same Data
0.9891891891891892


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
from __future__ import division
import pickle
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import average_precision_score
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


#Initialize stop words for Bahasa Indonesia
id_stop_words_file = pd.read_csv('id-stopwords.txt', header=None, names=['stopwords'])
id_stop_words_list = []
for i in range(len(id_stop_words_file)):
    id_stop_words_list.append(id_stop_words_file.values[i][0])

#Initialize Dataset
dataset = pd.read_csv('rangkuman-saja-copy.csv', header=0, sep=',', encoding='latin')
input = dataset['judul_proposal'].str.lower().str.replace('[^a-zA-Z0-9 ]', '')
target = dataset['deskripsi'].str.lower()

#Initialize Stemmer, Stemmernya cukup lama jadi comment aja biar cepet
#factory = StemmerFactory()
#stemmer = factory.create_stemmer()
#for i in range(len(input)):
#    print i
#    input[i] = stemmer.stem(input[i])

#Initialize TFIDF Vectorizer
tvect = TfidfVectorizer(min_df=1,stop_words=id_stop_words_list)

#Split Test dan Data Train
x_train, x_test, y_train, y_test = train_test_split(input, target, test_size=0.2, random_state=4)

tvect1 = TfidfVectorizer(min_df=1,stop_words=id_stop_words_list)
x_traincv=tvect1.fit_transform(x_train)
x_testcv=tvect1.transform(x_test)
x_traincv=tvect1.fit_transform(x_train)
x_testcv=tvect1.transform(x_test)


filename = 'KlasifikasiPakar.sav'
loaded_model = pickle.load(open(filename, 'rb'))

y_train = y_train.astype('str')
loaded_model.fit(x_traincv,y_train)

print("Cross Validation")
predictions=loaded_model.predict(x_testcv)
count = 0
for i in range (len(predictions)):
  if predictions[i]==y_test.values[i]:
     count = count + 1
print(count / len(y_test))


print("Same Data")
predictions=loaded_model.predict(x_traincv)
count = 0
for i in range (len(predictions)):
  if predictions[i]==y_train.values[i]:
     count = count + 1
print(count / len(y_train))

# Contoh 1 data
x = ['']
x_new=tvect1.transform(x)
new_pred = loaded_model.predict(x_new)
print(new_pred)

filename = 'tvect.sav'
pickle.dump(tvect1, open(filename, 'wb'))

Cross Validation
0.9435897435897436
Same Data
0.9948717948717949
['social and professional issue']
